# TBL data reduction

In [ ]:
import plopp as pp
from ess.reduce import time_of_flight
from ess import tbl
import ess.tbl.data  # noqa: F401
from ess.imaging.types import *

## Create and configure the workflow

In [ ]:
wf = tbl.TblWorkflow()

wf[Filename[SampleRun]] = tbl.data.tutorial_sample_data()
wf[TimeOfFlightLookupTableFilename] = tbl.data.tbl_tof_lookup_table_no_choppers()

## nGEM detector data

In [ ]:
wf[NeXusDetectorName] = "ngem_detector"

wf.visualize(DetectorData[SampleRun], graph_attr={"rankdir": "LR"})

In [ ]:
ngem = wf.compute(DetectorData[SampleRun])
ngem

In [ ]:
ngem['dim_0', 0].hist().plot(aspect='equal', title='First detection') + ngem['dim_0', 1].hist().plot(aspect='equal', title='Last detection')

### Time-of-flight

In [ ]:
wf.visualize(DetectorTofData[SampleRun], graph_attr={"rankdir": "LR"})

#### Visualize the time-of-flight lookup table

In [ ]:
table = wf.compute(time_of_flight.TimeOfFlightLookupTable)
table.plot()

#### Results

In [ ]:
ngem_tofs = wf.compute(DetectorTofData[SampleRun])
ngem_wavs = wf.compute(CountsWavelength[SampleRun])

ngem_tofs.bins.concat().hist(tof=100).plot() + ngem_wavs.bins.concat().hist(wavelength=100).plot()

## He3 detector data

In [ ]:
he3_wf = wf.copy()

he3 = {}
for bank in ('he3_detector_bank0', 'he3_detector_bank1'):
    he3_wf[NeXusDetectorName] = bank
    he3[bank] = he3_wf.compute(DetectorData[SampleRun])

In [ ]:
lims = {'vmin': 0, 'vmax': 4500}

he3['he3_detector_bank0'].hist().plot(title='he3_bank0', **lims) + he3['he3_detector_bank1'].hist().plot(title='he3_bank1', **lims)

### Time-of-flight

In [ ]:
he3_tofs = {}
he3_wavs = {}

for bank in ('he3_detector_bank0', 'he3_detector_bank1'):
    he3_wf[NeXusDetectorName] = bank
    he3_tofs[bank] = he3_wf.compute(DetectorTofData[SampleRun]).bins.concat().hist(tof=100)
    he3_wavs[bank] = he3_wf.compute(CountsWavelength[SampleRun]).bins.concat().hist(wavelength=100)

pp.plot(he3_tofs) + pp.plot(he3_wavs)